In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense


In [15]:
##Mapping of words index 
word_index = imdb.get_word_index()
reverse_word_index = {value :key for key , value in word_index.items()}

In [16]:
###Load the model
model = tf.keras.models.load_model('Simple_RNN_imdb.h5')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 2500, 128)      │     3,840,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,873,027 (14.77 MB)

 Trainable params: 3,873,025 (14.77 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [17]:
model.get_weights()

[array([[-0.04784405,  0.05354284, -0.0399468 , ...,  0.0136449 ,
          0.02400948, -0.03436881],
        [-0.02740226,  0.04749057, -0.03352843, ...,  0.00513046,
          0.02135923,  0.02286039],
        [ 0.0063851 ,  0.09629676,  0.01468128, ...,  0.02598275,
         -0.04417228,  0.03046189],
        ...,
        [-0.01240644,  0.07225703, -0.03861852, ..., -0.03077868,
         -0.07364631,  0.05678886],
        [-0.13928881, -0.17264102, -0.08304618, ...,  0.06530827,
         -0.13422492,  0.03600203],
        [ 0.0027373 , -0.06053567,  0.03757445, ...,  0.0023441 ,
          0.00938721,  0.00188647]], shape=(30000, 128), dtype=float32),
 array([[ 0.0956552 , -0.04654504, -0.07512464, ..., -0.15252367,
         -0.1225533 , -0.05803705],
        [-0.11639975, -0.0491041 , -0.08950243, ...,  0.02270527,
          0.02540044, -0.04216105],
        [-0.06863653,  0.13346988, -0.11719614, ..., -0.07432747,
         -0.05308507,  0.06834874],
        ...,
        [-0.1044547

In [18]:
###Step 2-helper Function
def decode_review(endcoded_review):
    return ' '.join([reverse_word_index.get(i-3, '?') for i in endcoded_review])

def preprocess_review(text):
    words = text.lower().split()
    endoded_review =[word_index.get(word,2)+3 for word in words]
    padded_review = sequence.pad_sequences([endoded_review], maxlen=500)
    return padded_review


In [9]:
###Predicton function

def predict_sentiment(review):
    preprocess = preprocess_review(review)
    models_prediction = model.predict(preprocess)
    
    sentiment = 'Positive' if models_prediction[0][0]>0.5 else 'Negative'
    return sentiment, float(models_prediction[0][0])

In [19]:
example_review = "This movie was just amazing, with stunning visuals and a gripping storyline that kept me on the edge of my seat from start to finish."

print(predict_sentiment(example_review))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 853ms/step
('Positive', 0.8215692043304443)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 853ms/step
('Positive', 0.8215692043304443)


In [20]:
# Test the problematic review that should be positive but shows as negative
problematic_review = "This movie was absolutely amazing! The acting was superb and the story was captivating."

print("Testing problematic review:")
print(f"Review: {problematic_review}")

# Check word recognition
words = problematic_review.lower().split()
print(f"Words: {words}")

# Check which words are in vocabulary
recognized_words = []
unknown_words = []

for word in words:
    clean_word = word.strip('!.,')  # Remove punctuation
    if clean_word in word_index:
        recognized_words.append(clean_word)
    else:
        unknown_words.append(clean_word)

print(f"Recognized words: {recognized_words}")
print(f"Unknown words: {unknown_words}")

# Get prediction
result = predict_sentiment(problematic_review)
print(f"Prediction: {result}")

# Let's also test if 'loved' is in the vocabulary
print(f"\nIs 'loved' in vocabulary? {'loved' in word_index}")
print(f"Is 'love' in vocabulary? {'love' in word_index}")
if 'love' in word_index:
    print(f"'love' index: {word_index['love']}")
if 'loved' in word_index:
    print(f"'loved' index: {word_index['loved']}")

Testing problematic review:
Review: This movie was absolutely amazing! The acting was superb and the story was captivating.
Words: ['this', 'movie', 'was', 'absolutely', 'amazing!', 'the', 'acting', 'was', 'superb', 'and', 'the', 'story', 'was', 'captivating.']
Recognized words: ['this', 'movie', 'was', 'absolutely', 'amazing', 'the', 'acting', 'was', 'superb', 'and', 'the', 'story', 'was', 'captivating']
Unknown words: []
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Prediction: ('Positive', 0.7123380899429321)

Is 'loved' in vocabulary? True
Is 'love' in vocabulary? True
'love' index: 116
'loved' index: 444
Prediction: ('Positive', 0.7123380899429321)

Is 'loved' in vocabulary? True
Is 'love' in vocabulary? True
'love' index: 116
'loved' index: 444
